<a href="https://colab.research.google.com/github/dowrave/DL_Models/blob/main/GoogleNet_InceptionV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import math



def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3,
                     filters_5x5_reduce, filters_5x5, filters_pool_proj, name = None,
                     kernel_init = 'glorot_uniform', bias_init = 'zeros'):
  
  conv_1x1 = layers.Conv2D(filters_1x1, (1,1), padding = 'same', activation = 'relu',
                           kernel_initializer = kernel_init, bias_initializer = bias_init)(x)
                  
  conv_3x3_reduce = layers.Conv2D(filters_3x3_reduce, (1,1), padding = 'same', activation = 'relu',
                           kernel_initializer = kernel_init, bias_initializer = bias_init)(x)
                  
  conv_3x3 = layers.Conv2D(filters_3x3, (3, 3), padding = 'same', activation = 'relu',
                           kernel_initializer = kernel_init, bias_initializer = bias_init)(conv_3x3_reduce)

  conv_5x5_reduce = layers.Conv2D(filters_3x3, (1,1), padding = 'same', activation = 'relu',
                           kernel_initializer = kernel_init, bias_initializer = bias_init)(x)

  conv_5x5 = layers.Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', 
                           kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5_reduce)

  max_pool = layers.MaxPool2D((3,3), strides = (1, 1), padding = 'same')(x)

  pool_proj = layers.Conv2D(filters_pool_proj, (1, 1), padding = 'same', activation = 'relu', kernel_initializer = kernel_init,
                            bias_initializer = bias_init)(max_pool)

  output = layers.concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis = 3, name = name)

  return output


kernel_init = tf.keras.initializers.glorot_uniform()
bias_init = tf.keras.initializers.Constant(value = 0.2)

# 층 구성
input_layer = layers.Input(shape = (224, 224, 3))

# 층 1
x = layers.Conv2D(64, (7, 7), padding = 'same', strides = (2, 2), activation = 'relu', name = 'conv_1_7x7/2', kernel_initializer = kernel_init,
                  bias_initializer = bias_init)(input_layer)
x = layers.MaxPool2D((3,3) , strides = (2, 2), name = 'maxpool_1_3x3/2', padding = 'same')(x)

# 층 2
x = layers.Conv2D(192, (3, 3), padding = 'same', strides = (1, 1), activation = 'relu', name = 'conv_2_3x3/1', kernel_initializer = kernel_init,
                  bias_initializer = bias_init)(input_layer)
x = layers.MaxPool2D((3,3) , strides = (2, 2), name = 'maxpool_2_3x3/2', padding = 'same')(x)

# 층 3
x = inception_module(x, 64, 96, 128, 16, 32, 32, name = 'inception_3a', kernel_init = kernel_init, bias_init = bias_init)
x = inception_module(x, 128, 128, 192, 32, 96, 64, name = 'inception_3b', kernel_init = kernel_init, bias_init = bias_init)
x = layers.MaxPool2D((3, 3), strides = (2, 2), name = 'max_pool_3_3x3/2')(x)

# 층 4
x = inception_module(x, 192, 96, 208, 16, 48, 64, name = 'inception_4a')

# 층 4 - Auxiliary Learning 1
x1 = layers.AveragePooling2D((5, 5), strides = 3, name = 'avg_pool_aux_1')(x)
x1 = layers.Conv2D(128, (1, 1), padding = 'same', activation = 'relu', name = 'conv_aux_1')(x1)
x1 = layers.Flatten()(x1)
x1 = layers.Dense(1024, activation = 'relu', name = 'dense_aux_1')(x1) 
x1 = layers.Dropout(0.7)(x1)
x1 = layers.Dense(num_classes, activation = 'softmax', name = 'aux_output_1')(x1)

x = inception_module(x, 160, 112, 224, 24, 64, 64, name = 'inception_4b', kernel_init = kernel_init, bias_init = bias_init)
x = inception_module(x, 128, 128, 256, 24, 64, 64, name = 'inception_4c', kernel_init = kernel_init, bias_init = bias_init)
x = inception_module(x, 112, 144, 288, 32, 64, 64, name = 'inception_4d', kernel_init = kernel_init, bias_init = bias_init)

# 층 4 - Auxiliary Learning 2
x2 = layers.AveragePooling2D((5, 5), strides = 3, name = 'avg_pool_aux_2')(x)
x2 = layers.Conv2D(128, (1, 1), padding = 'same', activation = 'relu', name = 'conv_aux_2')(x2)
x2 = layers.Flatten()(x2)
x2 = layers.Dense(1024, activation = 'relu', name = 'dense_aux_2')(x2) 
x2 = layers.Dropout(0.7)(x2)
x2 = layers.Dense(num_classes, activation = 'softmax', name = 'aux_output_2')(x2)

x = inception_module(x, 256, 160, 320, 32, 128, 128, name = 'inception_4e', kernel_init = kernel_init, bias_init = bias_init)
x = layers.MaxPool2D((3,3), strides = (2, 2), name = 'max_pool_4_3x3/2')(x)

# 층 5
x = inception_module(x, 256, 160, 320, 32, 128, 128, name = 'inception_5a', kernel_init = kernel_init, bias_init = bias_init)
x = inception_module(x, 384, 192, 384, 48, 128, 128, name = 'inception_5b', kernel_init = kernel_init, bias_init = bias_init)
x = layers.GlobalAveragePooling2D(name = 'global_avg_pool_5_3x3/1')(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(num_classes, activation = 'softmax', name= 'output')(x)

model = tf.keras.Model(input_layer, [x, x1, x2], name = 'inception_v1')

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epoch_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epoch_drop))
    return lrate

epoch = 25
initial_lrate = 0.01

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)
lr_sc = LearningRateScheduler(decay, verbose=1)


model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'],
              loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

history = model.fit(x_train, [y_train, y_train, y_train], validation_data=(x_valid, [y_valid, y_valid, y_valid]),
                    epochs=epoch, batch_size=20, callbacks=[lr_sc])